In [6]:
import json
import pennylane as qml
import pennylane.numpy as np

dev = qml.device("default.qubit", wires=3)

@qml.qnode(dev)
def model(alpha):
    x = alpha / 10
    
    # Adjust initial condition to avoid superposition at alpha=0
    theta_0 = np.pi * np.heaviside(np.sin(2 * np.pi * x + np.pi/4), 0.5)
    theta_1 = np.pi * np.heaviside(np.sin(4 * np.pi * x + np.pi/4), 0.5)
    theta_2 = np.pi * np.heaviside(np.sin(8 * np.pi * x + np.pi/4), 0.5)
    
    qml.RY(theta_0, wires=0)
    qml.RY(theta_1, wires=1)
    qml.RY(theta_2, wires=2)
    
    qml.CNOT(wires=[0, 1])
    qml.CNOT(wires=[1, 2])
    
    return qml.probs(wires=[0, 1, 2])

def generate_coefficients():
    # Map each index to the position where its state currently appears
    state_map = {
        0: 8.750,  # |000⟩ appears at 8.750
        1: 7.500,  # |001⟩ appears at 7.500
        2: 5.000,  # |010⟩ appears at 5.000
        3: 6.250,  # |011⟩ appears at 6.250
        4: 1.250,  # |100⟩ appears at 1.250
        5: 0.000,  # |101⟩ appears at 0.000
        6: 2.500,  # |110⟩ appears at 2.500
        7: 3.750   # |111⟩ appears at 3.750
    }
    return [state_map[i] for i in range(8)]
    
def test_solution():
    coefs = generate_coefficients()
    outputs = [model(alpha) for alpha in coefs]

    for i, probs in enumerate(outputs):
        # Find the index of the maximum probability
        state_index = np.argmax(probs)
        # Convert index to binary representation
        state_binary = format(state_index, '03b')
        print(f"Alpha: {coefs[i]:.3f}, State: |{state_binary}⟩, Probabilities: {probs}")

test_solution()


Alpha: 8.750, State: |000⟩, Probabilities: [1. 0. 0. 0. 0. 0. 0. 0.]
Alpha: 7.500, State: |001⟩, Probabilities: [3.74939946e-33 1.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
Alpha: 5.000, State: |010⟩, Probabilities: [1.40579963e-65 3.74939946e-33 1.00000000e+00 3.74939946e-33
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
Alpha: 6.250, State: |011⟩, Probabilities: [3.74939946e-33 0.00000000e+00 0.00000000e+00 1.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
Alpha: 1.250, State: |100⟩, Probabilities: [1.40579963e-65 0.00000000e+00 0.00000000e+00 3.74939946e-33
 1.00000000e+00 0.00000000e+00 0.00000000e+00 3.74939946e-33]
Alpha: 0.000, State: |101⟩, Probabilities: [5.27090436e-98 1.40579963e-65 3.74939946e-33 1.40579963e-65
 3.74939946e-33 1.00000000e+00 3.74939946e-33 1.40579963e-65]
Alpha: 2.500, State: |110⟩, Probabilities: [1.40579963e-65 3.74939946e-33 0.00000000e+00 0.00000000e+